# Lecture 12 状態空間モデル

## ローカルレベルモデルを試してみる

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import io
import requests
import statsmodels.api as sm

In [ ]:

# 月ごとの飛行機の乗客数データ
url = "https://www.analyticsvidhya.com/wp-content/uploads/2016/02/AirPassengers.csv"
stream = requests.get(url).content
df = pd.read_csv(io.StringIO(stream.decode('utf-8')),
                   index_col="Month",
                   parse_dates=True,
                   dtype="float")
df.head()

In [ ]:
df.plot()

In [ ]:
df.columns

In [ ]:
# 学習と評価データに分割
df_train = df[df.index < '1957-04-01']
df_test = df[df.index >= '1957-04-01']

In [ ]:


# local level model
local_level_model = sm.tsa.UnobservedComponents(df_train, 'llevel')
# parameter estimation with maximum likelihood estimation
local_level_params = local_level_model.fit(
                        method='bfgs',
                        maxiter=500)
# plot summary
fig = local_level_params.plot_components()

In [ ]:
st = pd.to_datetime(df_test.index[0])
ed = pd.to_datetime(df_test.index[-1])
# forecast future values
local_level_pred = local_level_params.predict(st, ed)
# plot results
ax1 = df_test.plot(figsize=(16,4))
df_test.plot(color="0.7",
          linestyle='dotted',
          linewidth="5.0",
          ax=ax1)
#sarima_pred.plot(ax=ax1)
local_level_pred.plot(ax=ax1)
#plt.legend(['train', 'original', 'SARIMA', 'State Space'])
plt.legend(['train', 'original', 'State Space'])
plt.xlabel('Duration (month)')
plt.ylabel('# of Passengers')
plt.show()

In [ ]:
# local level model with seasonality
seasonal_model = sm.tsa.UnobservedComponents(df_train, 'lltrend', seasonal=12)
# parameter estimation with maximum likelihood estimation
seasonal_params = seasonal_model.fit(
                        method='bfgs',
                        maxiter=500)
# plot summary
fig = seasonal_params.plot_components()

In [ ]:
st = pd.to_datetime(df_test.index[0])
ed = pd.to_datetime(df_test.index[-1])
# forecast future values
statespace_pred = seasonal_params.predict(st, ed)
# plot results
ax1 = df_test.plot(figsize=(16,4))
df_test.plot(color="0.7",
          linestyle='dotted',
          linewidth="5.0",
          ax=ax1)
#sarima_pred.plot(ax=ax1)
statespace_pred.plot(ax=ax1)
#plt.legend(['train', 'original', 'SARIMA', 'State Space'])
plt.legend(['train', 'original', 'State Space'])
plt.xlabel('Duration (month)')
plt.ylabel('# of Passengers')
plt.show()

## ローカルレベルモデルで様々なモデル（トレンドや季節性の取り込みや分散の省略等）を試して精度を比較する。

参考：https://logics-of-blue.com/wp-content/uploads/2017/05/python-state-space-models.html



|	|model|	説明
|---|---|---|
|0	|'local level'|ローカルレベルモデル
|1	|'local linear trend'|ローカル線形トレンドモデル
|2	|'local level',seasonal=12|季節変動ありローカルレベルモデル
|3	|'local linear trend',seasonal=12|	季節変動ありローカル線形トレンドモデル
|4	|'local linear deterministic trend',seasonal=12|季節変動ありローカル線形トレンドモデル(トレンド分散なし)
|5	| 'random walk with drift',seasonal=12|季節変動ありローカル線形トレンドモデル(トレンド分散なし、観測誤差なし

In [ ]:
# 基本のライブラリを読み込む
import numpy as np
import pandas as pd
from scipy import stats

# グラフ描画
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib inline

# グラフを横長にする
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

# 統計モデル
import statsmodels.api as sm

In [ ]:
# プロット
plt.plot(df)

ローカルレベルモデルの推定


In [ ]:
# ローカルレベルモデルの推定
mod_local_level = sm.tsa.UnobservedComponents(df, 'local level')

# 最尤法によるパラメタの推定
res_local_level = mod_local_level.fit()

# 推定されたパラメタ一覧
print(res_local_level.summary())

# 推定された状態・トレンドの描画
rcParams['figure.figsize'] = 15, 15
fig = res_local_level.plot_components()

ローカル線形トレンドモデルの推定

In [ ]:
# ローカル線形トレンドモデル

mod_trend = sm.tsa.UnobservedComponents(
    df,
    'local linear trend'
)

# 最尤法によるパラメタの推定
# ワーニングが出たのでBFGS法で最適化する
res_trend = mod_trend.fit(method='bfgs')

# 推定されたパラメタ一覧
print(res_trend.summary())

# 推定された状態・トレンドの描画
rcParams['figure.figsize'] = 15, 20
fig = res_trend.plot_components()

季節変動の取り込み


In [ ]:
# 季節変動ありのローカルレベルモデル

mod_season_local_level = sm.tsa.UnobservedComponents(
    df,
    'local level',
    seasonal=12)

# まずはNelder-Meadでパラメタを推定し、その結果を初期値としてまた最適化する。2回目はBFGSを使用。
res_season_local_level = mod_season_local_level.fit(
    method='bfgs',
    maxiter=500,
    start_params=mod_season_local_level.fit(method='nm', maxiter=500).params,
)

# 推定されたパラメタ一覧
print(res_season_local_level.summary())

# 推定された状態・トレンド・季節の影響の描画
rcParams['figure.figsize'] = 15, 20
fig = res_season_local_level.plot_components()

In [ ]:
# 季節変動ありのローカル線形トレンドモデル

mod_season_trend = sm.tsa.UnobservedComponents(
    df,
    'local linear trend',
    seasonal=12)

# まずはNelder-Meadでパラメタを推定し、その結果を初期値としてまた最適化する。2回目はBFGSを使用。
res_season_trend = mod_season_trend.fit(
    method='bfgs',
    maxiter=500,
    start_params=mod_season_trend.fit(method='nm', maxiter=500).params,
)

# 推定されたパラメタ一覧
print(res_season_trend.summary())

# 推定された状態・トレンド・季節の影響の描画
rcParams['figure.figsize'] = 15, 20
fig = res_season_trend.plot_components()

推定するパラメタの数を減らす

In [ ]:
# 詳細は以下の資料を参照してください
# http://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.structural.UnobservedComponents.html#statsmodels.tsa.statespace.structural.UnobservedComponents

# 季節変動ありのローカル線形トレンドモデル
# ただし、トレンドの分散は無し

mod_season_trend_d = sm.tsa.UnobservedComponents(
    df,
    'local linear deterministic trend',
    seasonal=12)

# まずはNelder-Meadでパラメタを推定し、その結果を初期値としてまた最適化する。2回目はBFGSを使用。
res_season_trend_d = mod_season_trend_d.fit(
    method='bfgs',
    maxiter=500,
    start_params=mod_season_trend_d.fit(method='nm', maxiter=500).params,
)

# 推定されたパラメタ一覧
print(res_season_trend_d.summary())

# 推定された状態・トレンド・季節の影響の描画
rcParams['figure.figsize'] = 15, 20
fig = res_season_trend_d.plot_components()

In [ ]:
# 詳細は以下の資料を参照してください
# http://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.structural.UnobservedComponents.html#statsmodels.tsa.statespace.structural.UnobservedComponents

# 季節変動ありのローカル線形トレンドモデル
# ただし、トレンドの分散は無し

mod_season_rw = sm.tsa.UnobservedComponents(
    df,
    'random walk with drift',
    seasonal=12)

# まずはNelder-Meadでパラメタを推定し、その結果を初期値としてまた最適化する。2回目はBFGSを使用。
res_season_rw = mod_season_rw.fit(
    method='bfgs',
    maxiter=500,
    start_params=mod_season_rw.fit(method='nm', maxiter=500).params,
)

# 推定されたパラメタ一覧
print(res_season_rw.summary())

# 推定された状態・トレンド・季節の影響の描画
rcParams['figure.figsize'] = 15, 20
fig = res_season_rw.plot_components()

In [ ]:
# 今まで計算してきたモデルのAICを格納する
aic_list = pd.DataFrame(index=range(6), columns=["model", "aic"])

aic_list.iloc[0]["model"] = "res_local_level"
aic_list.iloc[0]["aic"] = res_local_level.aic

aic_list.iloc[1]["model"] = "res_trend"
aic_list.iloc[1]["aic"] = res_trend.aic

aic_list.iloc[2]["model"] = "res_season_local_level"
aic_list.iloc[2]["aic"] = res_season_local_level.aic

aic_list.iloc[3]["model"] = "res_season_trend"
aic_list.iloc[3]["aic"] = res_season_trend.aic

aic_list.iloc[4]["model"] = "res_season_trend_d"
aic_list.iloc[4]["aic"] = res_season_trend_d.aic

aic_list.iloc[5]["model"] = "res_season_rw"
aic_list.iloc[5]["aic"] = res_season_rw.aic


# 結果の表示
aic_list

In [ ]:
# 予測
pred = res_season_rw.predict('1960-01-01', '1961-12-01')

# 実データと予測結果の図示
rcParams['figure.figsize'] = 15, 6
plt.plot(df)
plt.plot(pred, "r")
plt.title('res_season_rw')

In [ ]:
# 予測
pred = res_local_level.predict('1960-01-01', '1961-12-01')

# 実データと予測結果の図示
rcParams['figure.figsize'] = 15, 6
plt.plot(df)
plt.plot(pred, "r")
plt.title('res_local_level')

In [ ]:
# 予測
pred = res_trend.predict('1960-01-01', '1961-12-01')

# 実データと予測結果の図示
rcParams['figure.figsize'] = 15, 6
plt.plot(df)
plt.plot(pred, "r")
plt.title('res_trend')

In [ ]:
# 予測
pred = res_season_local_level.predict('1960-01-01', '1961-12-01')

# 実データと予測結果の図示
rcParams['figure.figsize'] = 15, 6
plt.plot(df)
plt.plot(pred, "r")
plt.title('res_season_local_level')

## 線形ガウス型モデルの設計と解析([島田])

- Pycalmanによるトレンドの推定モデル構築

In [ ]:
!pip install pykalman

In [ ]:
%matplotlib inline
import numpy as np
np.random.seed(555)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
import statsmodels as sm
from statsmodels.graphics import tsaplots
from statsmodels.tsa import stattools
from pykalman import KalmanFilter

In [ ]:
# 月ごとの飛行機の乗客数データ
url = "https://www.analyticsvidhya.com/wp-content/uploads/2016/02/AirPassengers.csv"
df.head()

In [ ]:
df.plot()

In [ ]:
# 推移行列などの初期化
## ここで n_dim_obs=1, n_dim_trend=2とする
def FGHset(n_dim_trend, n_dim_obs=1, n_dim_series=0, Q_sigma2=10):
    n_dim_Q=2
    n_dim_state = n_dim_trend

    # 行列の初期化
    G = np.zeros((n_dim_state, n_dim_Q))
    F = np.zeros((n_dim_state, n_dim_state))
    H = np.zeros((n_dim_obs, n_dim_state))
    Q = np.eye(n_dim_Q) * Q_sigma2

    # 各行列のトレンド成分に対するブロック行列を構築
    G[0,0] = 1
    H[0,0] = 1

    # トレンドモデルの推移行列の構築
    # 非定常過程でも対応できる推移行列を構築
    F[0,0] = 2
    F[0,1] = -1
    F[1,0] = 1

    # PyKalmanのQはG.dot(Q).dot(G.T)を想定しているためQをupdate
    Q = G.dot(Q).dot(G.T)

    return n_dim_state, F, H, Q

In [ ]:
# 観測値の次元数
n_dim_obs = 1
# トレンドの次元数
n_dim_trend = 2

# 推移行列などの定義
n_dim_state, F, H, Q = FGHset(n_dim_trend, n_dim_obs)

In [ ]:
 n_dim_state, F, H, Q

In [ ]:
# 状態の平均値ベクトルの初期値
initial_state_mean = np.zeros(n_dim_state)
# 状態の分散共分散行列の初期値
initial_state_covariance = np.ones((n_dim_state, n_dim_state))

# カルマンフィルタのモデル生成
kf = KalmanFilter(
    # l：観測値の次元数
    n_dim_obs=n_dim_obs,
    # k：状態の次元数
    n_dim_state=n_dim_state,
    # x_0：状態の平均値ベクトルの初期値（k次元）
    initial_state_mean=initial_state_mean,
    # V_0：状態の分散共分散行列の初期値（k×k次元）
    initial_state_covariance=initial_state_covariance,
    # F：推移行列（k×k次元）
    transition_matrices=F,
    # H：観測行列（l×k次元）
    observation_matrices=H,
    # R：観測ノイズwの分散共分散行列（l×l次元。観測値が一次元の場合はスカラ）
    observation_covariance=1.0,
    # Q：システムノイズvの分散共分散行列（m×m次元）
    transition_covariance=Q)

In [ ]:
# 前半120時点を学習データに121時点以降のデータを検証用に使用
n_train = 120
train, test = df.values[:n_train], df.values[n_train:]

In [ ]:
smoothed_state_means, smoothed_state_covs = kf.smooth(train)
pred_o_smoothed = smoothed_state_means.dot(H.T)

In [ ]:
plt.plot(train, label="observation")
plt.plot(pred_o_smoothed, '--', label="predict")

In [ ]:
plt.plot(df.values, label="observation")

# 長期予測格納用のベクトルを用意（値は全て入れ替わるため0で初期化しなくてよい）
pred_y = np.empty(len(test))

# 現在の状態と分散共分散行列を取得
current_state = smoothed_state_means[-1]
current_cov = smoothed_state_covs[-1]
for i in range(len(test)):
    # filter_updateは観測値を入力しなければ1期先予測のみを実行する
    current_state, current_cov = kf.filter_update(current_state,
                                                  current_cov,
                                                  observation=None)
    pred_y[i] = kf.observation_matrices.dot(current_state)

# np.hstackはnp.concatenateでもよい
plt.plot(np.hstack([pred_o_smoothed.flatten(), pred_y]), '--', label="forecast")

### Kalman by seasonal

In [ ]:
# 推移行列などの初期化
def FGHset(n_dim_trend, n_dim_obs=1, n_dim_series=0, Q_sigma2=10):
    n_dim_Q = (n_dim_trend!=0) + (n_dim_series!=0)
    if n_dim_series>0:
        n_dim_state = n_dim_trend + n_dim_series - 1
    else:
        n_dim_state = n_dim_trend

    # 行列の初期化
    G = np.zeros((n_dim_state, n_dim_Q))
    F = np.zeros((n_dim_state, n_dim_state))
    H = np.zeros((n_dim_obs, n_dim_state))
    Q = np.eye(n_dim_Q) * Q_sigma2

    ## トレンドモデルのブロック行列の構築
    G[0,0] = 1
    H[0,0] = 1
    if n_dim_trend==1:
        F[0,0] = 1
    elif n_dim_trend==2:
        F[0,0] = 2
        F[0,1] = -1
        F[1,0] = 1
    elif n_dim_trend==3:
        F[0,0] = 3
        F[0,1] = -3
        F[0,2] = 1
        F[1,0] = 1
        F[2,1] = 1

    start_elem = n_dim_trend
    start_col = n_dim_trend
    # 季節調整成分のブロック行列の構築
    if n_dim_series>0:
        G[start_elem, 1] = 1
        H[0, start_elem] = 1
        for i in range(n_dim_series-1):
            F[start_elem, start_elem+i] = -1
        for i in range(n_dim_series-2):
            F[start_elem+i+1, start_elem+i] = 1

    # PyKalmanのQはG.dot(Q).dot(G.T)を想定しているためQをupdate
    Q = G.dot(Q).dot(G.T)

    return n_dim_state, F, H, Q

In [ ]:
# 観測値の次元数
n_dim_obs = 1
# トレンドの次元数
n_dim_trend = 2
# 季節成分の次元数
n_dim_series = 12

# 推移行列などの定義
n_dim_state, F, H, Q = FGHset(n_dim_trend, n_dim_obs, n_dim_series)

In [ ]:
# 状態の平均値ベクトルの初期値
initial_state_mean = np.zeros(n_dim_state)
# 状態の分散共分散行列の初期値
initial_state_covariance = np.ones((n_dim_state, n_dim_state))

# カルマンフィルタのモデル生成
kf = KalmanFilter(
    # l：観測値の次元数
    n_dim_obs=n_dim_obs,
    # k：状態の次元数
    n_dim_state=n_dim_state,
    # x_0：状態の平均値ベクトルの初期値（k次元）
    initial_state_mean=initial_state_mean,
    # V_0：状態の分散共分散行列の初期値（k×k次元）
    initial_state_covariance=initial_state_covariance,
    # F：推移行列（k×k次元）
    transition_matrices=F,
    # H：観測行列（l×k次元）
    observation_matrices=H,
    # R：観測ノイズwの分散共分散行列（l×l次元。観測値が一次元の場合はスカラ）
    observation_covariance=1.0,
    # Q：システムノイズvの分散共分散行列（m×m次元）
    transition_covariance=Q)

In [ ]:
# 前半120時点を学習データに121時点以降のデータを検証用に使用
n_train = 120
train, test = df.values[:n_train], df.values[n_train:]

In [ ]:
# フィルタ＋平滑化
smoothed_state_means, smoothed_state_covs = kf.smooth(train)
pred_o_smoothed = smoothed_state_means.dot(H.T)

In [ ]:
plt.plot(train, label="observation")
plt.plot(pred_o_smoothed, '--', label="predict")

In [ ]:
plt.plot(df.values, label="observation")

pred_y = np.empty(len(test))
current_state = smoothed_state_means[-1]
current_cov = smoothed_state_covs[-1]
for i in range(len(test)):
    current_state, current_cov = kf.filter_update(current_state,
                                                  current_cov,
                                                  observation=None)
    pred_y[i] = kf.observation_matrices.dot(current_state)

plt.plot(np.hstack([pred_o_smoothed.flatten(), pred_y]), '--', label="forecast")

## EMアルゴリズムを用いたハイパーパラメタ最適化

In [ ]:
# 繰り返し回数は10回
# ハイパーパラメタ更新の対象はF, H, Q, R
emed_kf = kf.em(train, n_iter=10, em_vars='all')

In [ ]:
# ハイパーパラメタF, H, Q, R更新後の平滑化系列
em_smoothed_state_means, em_smoothed_state_covs = emed_kf.smooth(train)
em_pred_o_smoothed = np.dot(em_smoothed_state_means, emed_kf.observation_matrices.T)
plt.plot(df.values, label="observation")

current_state = em_smoothed_state_means[-1]
current_cov = em_smoothed_state_covs[-1]

pred_y = np.empty(len(test))
for i in range(len(test)):
    current_state, current_cov = emed_kf.filter_update(current_state, current_cov, observation=None)
    pred_y[i] = kf.observation_matrices.dot(current_state)

plt.plot(np.hstack([em_pred_o_smoothed.flatten(), pred_y]), '--', label="forecast")

In [ ]:
emed_kf.transition_matrices

### seasonal + AR

In [ ]:
# 推移行列などの初期化
def FGHset(n_dim_trend,
           n_dim_obs=1, n_dim_series=0, n_dim_ar=0, Q_sigma2=10):
    n_dim_Q = (n_dim_trend!=0) + (n_dim_series!=0) + (n_dim_ar!=0)
    if n_dim_series>0 or n_dim_ar>0:
        n_dim_state = n_dim_trend + n_dim_series + n_dim_ar - 1
    else:
        n_dim_state = n_dim_trend

    # 行列の初期化
    G = np.zeros((n_dim_state, n_dim_Q))
    F = np.zeros((n_dim_state, n_dim_state))
    H = np.zeros((n_dim_obs, n_dim_state))
    Q = np.eye(n_dim_Q) * Q_sigma2

    ## トレンドモデルのブロック行列の構築
    G[0,0] = 1
    H[0,0] = 1
    if n_dim_trend==1:
        F[0,0] = 1
    elif n_dim_trend==2:
        F[0,0] = 2
        F[0,1] = -1
        F[1,0] = 1
    elif n_dim_trend==3:
        F[0,0] = 3
        F[0,1] = -3
        F[0,2] = 1
        F[1,0] = 1
        F[2,1] = 1

    start_elem = n_dim_trend
    start_col = n_dim_trend
    # 季節調整成分のブロック行列の構築
    if n_dim_series>0:
        G[start_elem, 1] = 1
        H[0, start_elem] = 1
        for i in range(n_dim_series-1):
            F[start_elem, start_elem+i] = -1
        for i in range(n_dim_series-2):
            F[start_elem+i+1, start_elem+i] = 1

        start_elem = n_dim_trend + n_dim_series - 1
        start_col = n_dim_trend + n_dim_series - 1

    # AR成分のブロック行列の構築
    if n_dim_ar>0:
        G[start_elem, 2] = 1
        H[0, start_elem] = 1
        for i in range(n_dim_ar):
            F[start_elem, start_elem+i] = 0
        for i in range(n_dim_ar-1):
            F[start_elem+i+1, start_elem+i] = 1


    # PyKalmanのQはG.dot(Q).dot(G.T)を想定しているためQをupdate
    Q = G.dot(Q).dot(G.T)

    return n_dim_state, F, H, Q

In [ ]:
# 観測値の次元数
n_dim_obs = 1
# トレンドの次元数
n_dim_trend = 2
# 季節成分の次元数
n_dim_series = 12
# AR成分の次元数
n_dim_ar = 2

# 推移行列などの定義
n_dim_state, F, H, Q = FGHset(n_dim_trend, n_dim_obs, n_dim_series, n_dim_ar)

In [ ]:
# 状態の平均値ベクトルの初期値
initial_state_mean = np.zeros(n_dim_state)
# 状態の分散共分散行列の初期値
initial_state_covariance = np.ones((n_dim_state, n_dim_state))

# カルマンフィルタのモデル生成
kf = KalmanFilter(
    # l：観測値の次元数
    n_dim_obs=n_dim_obs,
    # k：状態の次元数
    n_dim_state=n_dim_state,
    # x_0：状態の平均値ベクトルの初期値（k次元）
    initial_state_mean=initial_state_mean,
    # V_0：状態の分散共分散行列の初期値（k×k次元）
    initial_state_covariance=initial_state_covariance,
    # F：推移行列（k×k次元）
    transition_matrices=F,
    # H：観測行列（l×k次元）
    observation_matrices=H,
    # R：観測ノイズwの分散共分散行列（l×l次元。観測値が一次元の場合はスカラ）
    observation_covariance=1.0,
    # Q：システムノイズvの分散共分散行列（m×m次元）
    transition_covariance=Q)

In [ ]:
# フィルタ＋平滑化
smoothed_state_means, smoothed_state_covs = kf.smooth(train)
pred_o_smoothed = smoothed_state_means.dot(H.T)

In [ ]:
plt.plot(train, label="observation")
plt.plot(pred_o_smoothed, '--', label="predict")

In [ ]:
plt.plot(df.values, label="observation")

pred_y = np.empty(len(test))
current_state = smoothed_state_means[-1]
current_cov = smoothed_state_covs[-1]
for i in range(len(test)):
    current_state, current_cov = kf.filter_update(current_state,
                                                  current_cov,
                                                  observation=None)
    pred_y[i] = kf.observation_matrices.dot(current_state)

plt.plot(np.hstack([pred_o_smoothed.flatten(), pred_y]), '--', label="forecast")